## Wobbly Rosenbrock (parameter noise in 'a') : nominal vs. CVaR-optimized

In [1]:
import argparse
import numpy as np
import casadi as ca
import matplotlib.pyplot as plt


def rosenbrock(x, a, b):
    x1, x2 = x[0], x[1]
    return (a - x1) ** 2 + b * (x2 - x1**2) ** 2


def cvar_empirical(values, alpha: float):
    """
    Empirical CVaR_alpha for a 1D array of loss values:
      CVaR = mean of the upper (1-alpha) tail (>= VaR_alpha).
    """
    values = np.asarray(values, dtype=float)
    if not (0.0 < alpha < 1.0):
        raise ValueError("alpha must be in (0,1)")
    if values.size == 0:
        return np.nan
    # Empirical VaR at level alpha
    var_alpha = np.quantile(values, alpha)
    tail = values[values >= var_alpha]
    return float(tail.mean())


def solve_nominal(a=1.0, b=100.0, lbx=(-2.0, -2.0), ubx=(2.0, 2.0), x0=(-1.2, 1.0)):
    """Minimize the deterministic Rosenbrock (no noise)."""
    x = ca.MX.sym("x", 2)  # type: ignore
    f = rosenbrock(x, a, b)
    nlp = {"x": x, "f": f}
    solver = ca.nlpsol("nom", "ipopt", nlp, {"ipopt.print_level": 0, "print_time": 0})
    sol = solver(x0=x0, lbx=lbx, ubx=ubx)
    x_nom = np.array(sol["x"]).squeeze()
    f_nom = float(sol["f"])
    return x_nom, f_nom


def solve_cvar(
    a=1.0,
    b=100.0,
    alpha=0.95,
    Xi=None,
    lbx=(-2.0, -2.0),
    ubx=(2.0, 2.0),
    x0=(-1.2, 1.0),
):
    """
    Solve CVaR minimization using RU formulation with parameter noise in 'a'.
    Xi: array of shape (N,), zero-mean noise samples for 'a'.
    """
    assert Xi is not None and Xi.ndim == 1
    N = Xi.shape[0]

    x = ca.MX.sym("x", 2)  # type: ignore
    t = ca.MX.sym("t")  # type: ignore
    u = ca.MX.sym("u", N)  # type: ignore

    g = []
    for i in range(N):
        a_eff = a + float(Xi[i])
        fi = rosenbrock(x, a_eff, b)
        g.append(u[i] - (fi - t))  # u_i >= f - t
        g.append(u[i])  # u_i >= 0

    obj = t + (1.0 / ((1.0 - alpha) * N)) * ca.sum1(u)

    w = ca.vertcat(x, t, u)
    lbg = [0.0] * len(g)
    ubg = [ca.inf] * len(g)
    lbw = list(lbx) + [-ca.inf] + [0.0] * N
    ubw = list(ubx) + [ca.inf] + [ca.inf] * N
    w0 = np.r_[x0, 1.0, np.zeros(N)]

    nlp = {"x": w, "f": obj, "g": ca.vertcat(*g)}
    solver = ca.nlpsol("cvar", "ipopt", nlp, {"ipopt.print_level": 0, "print_time": 0})
    sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
    w_opt = np.array(sol["x"]).squeeze()
    x_cvar = w_opt[:2]
    t_cvar = w_opt[2]
    obj_cvar = float(sol["f"])
    return x_cvar, t_cvar, obj_cvar


def evaluate_losses(xval, a, b, Xi):
    """Compute array of losses f(x, xi_a) for samples Xi."""
    vals = np.array(
        [float((a + z - xval[0]) ** 2 + b * (xval[1] - xval[0] ** 2) ** 2) for z in Xi],
        dtype=float,
    )
    return vals


In [2]:
## Set the parameters for the experiment

a = 1.0
b = 100.0
alpha = 0.99
N = 500
sigma_a = 0.05
M = 5000
lbx, ubx = (-2.0, -2.0), (2.0, 2.0)
x0 = (-1.2, -1.0)

#### Normal Distribution Pertubation


In [3]:
seed = 144
dist = "normal"
rng = np.random.default_rng(seed)
Xi_train = rng.normal(0.0, sigma_a, size=N)
Xi_eval = rng.normal(0.0, sigma_a, size=M)

# Solve nominal problem (deterministic Rosenbrock with a=1.0, no noise)
x_nom, f_nom = solve_nominal(a=a, b=b, lbx=lbx, ubx=ubx, x0=x0)
losses_nom = evaluate_losses(x_nom, a, b, Xi_eval)
cvar_nom = cvar_empirical(losses_nom, alpha)
mean_nom = float(losses_nom.mean())
print("Nominal solution (deterministic Rosenbrock):")
print("  x_nom =", x_nom, "  f(x_nom; a,b) =", f_nom)
print(
    "  OOS (M=%d) mean=%.6f  CVaR@alpha=%.2f = %.6f"
    % (M, mean_nom, alpha, cvar_nom)
)
print()

## Solve CVaR problem with parameter noise in 'a'
x_cvar, t_cvar, obj_cvar = solve_cvar(
    a=a, b=b, alpha=alpha, Xi=Xi_train, lbx=lbx, ubx=ubx, x0=x0
)
losses_cvar = evaluate_losses(x_cvar, a, b, Xi_eval)
cvar_cvar = cvar_empirical(losses_cvar, alpha)
mean_cvar = float(losses_cvar.mean())
print("CVaR-optimized solution (train N=%d, alpha=%.2f):" % (N, alpha))
print(
    "  x_cvar =",
    x_cvar,
    "  t* =",
    t_cvar,
    "  training objective (RU) =",
    obj_cvar,
)
print(
    "  OOS (M=%d) mean=%.6f  CVaR@alpha=%.2f = %.6f"
    % (M, mean_cvar, alpha, cvar_cvar)
)
print()

## Compare nominal vs CVaR solutions
print("Comparison (OOS): CVaR@alpha=%.2f" % alpha)
print(
    "  CVaR(x_cvar)  vs  CVaR(x_nom)  -->  %.6f  vs  %.6f"
    % (cvar_cvar, cvar_nom)
)
print(
    "  Mean(x_cvar)  vs  Mean(x_nom)  -->  %.6f  vs  %.6f"
    % (mean_cvar, mean_nom)
)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Nominal solution (deterministic Rosenbrock):
  x_nom = [0.99999999 0.99999998]   f(x_nom; a,b) = 1.1434476595638644e-16
  OOS (M=5000) mean=0.002417  CVaR@alpha=0.99 = 0.020196

CVaR-optimized solution (train N=500, alpha=0.99):
  x_cvar = [0.99655025 0.99311239]   t* = 0.016393286591070193   training objective (RU) = 0.019955877173173212
  OOS (M=5000) mean=0.002417  CVaR@alpha=0.99 = 0.019941

Comparison (OOS): CVaR@alpha=0.99
  CVaR(x_cvar)  vs  CVaR(x_nom)  -->  0.019941  vs  0.020196
  Mean(x_cvar)  vs  Mean(x_nom)  -->  0.002417  vs  0.002417


### Students t distribution pertubation for heavy tails

In [4]:
seed = 340
dist = "students_t"
rng = np.random.default_rng(seed)
Xi_train = rng.standard_t(df=3, size=N) * sigma_a
Xi_eval = rng.standard_t(df=3, size=M) * sigma_a

# Solve nominal problem (deterministic Rosenbrock with a=1.0, no noise)
x_nom, f_nom = solve_nominal(a=a, b=b, lbx=lbx, ubx=ubx, x0=x0)
losses_nom = evaluate_losses(x_nom, a, b, Xi_eval)
cvar_nom = cvar_empirical(losses_nom, alpha)
mean_nom = float(losses_nom.mean())
print("Nominal solution (deterministic Rosenbrock):")
print("  x_nom =", x_nom, "  f(x_nom; a,b) =", f_nom)
print("  OOS (M=%d) mean=%.6f  CVaR@alpha=%.2f = %.6f" % (M, mean_nom, alpha, cvar_nom))
print()

## Solve CVaR problem with parameter noise in 'a'
x_cvar, t_cvar, obj_cvar = solve_cvar(
    a=a, b=b, alpha=alpha, Xi=Xi_train, lbx=lbx, ubx=ubx, x0=x0
)
losses_cvar = evaluate_losses(x_cvar, a, b, Xi_eval)
cvar_cvar = cvar_empirical(losses_cvar, alpha)
mean_cvar = float(losses_cvar.mean())
print("CVaR-optimized solution (train N=%d, alpha=%.2f):" % (N, alpha))
print(
    "  x_cvar =",
    x_cvar,
    "  t* =",
    t_cvar,
    "  training objective (RU) =",
    obj_cvar,
)
print(
    "  OOS (M=%d) mean=%.6f  CVaR@alpha=%.2f = %.6f" % (M, mean_cvar, alpha, cvar_cvar)
)
print()

## Compare nominal vs CVaR solutions
print("Comparison (OOS): CVaR@alpha=%.2f" % alpha)
print("  CVaR(x_cvar)  vs  CVaR(x_nom)  -->  %.6f  vs  %.6f" % (cvar_cvar, cvar_nom))
print("  Mean(x_cvar)  vs  Mean(x_nom)  -->  %.6f  vs  %.6f" % (mean_cvar, mean_nom))

Nominal solution (deterministic Rosenbrock):
  x_nom = [0.99999999 0.99999998]   f(x_nom; a,b) = 1.1434476595638644e-16
  OOS (M=5000) mean=0.007048  CVaR@alpha=0.99 = 0.237490

CVaR-optimized solution (train N=500, alpha=0.99):
  x_cvar = [0.95033942 0.90314501]   t* = 0.06865876555028139   training objective (RU) = 0.17558252015529915
  OOS (M=5000) mean=0.009584  CVaR@alpha=0.99 = 0.234932

Comparison (OOS): CVaR@alpha=0.99
  CVaR(x_cvar)  vs  CVaR(x_nom)  -->  0.234932  vs  0.237490
  Mean(x_cvar)  vs  Mean(x_nom)  -->  0.009584  vs  0.007048
